## Travel Time for Central America using GOSTnets Raster
Generates rasters for shortest travel time to health facilities

### Cities include: Costa Rica, Dominican Republic, Nicaragua, Guatemala, Panama, El Salvador

### Inputs:
    - origins: population grid
    - destinations: health facilities
    
Friction layer is the 2020 driving raster Global Friction Surface from the Malaria Access Project (https://malariaatlas.org/project-resources/accessibility-to-healthcare/)

## As of May 2023, use a python 3.8 environment, so that pandana can be installed

In [355]:
import sys, os, importlib
import rasterio

import numpy as np
import pandas as pd
import geopandas as gpd
import osmnx as ox

import skimage.graph as graph

from rasterio.mask import mask
from rasterio import features

from shapely.geometry import box, Point, Polygon
from scipy.ndimage import generic_filter
#from pandana.loaders import osm

sys.path.append("../")

In [356]:
sys.path.append(r"/Users/tomgertin/repos/GOSTnets")
import GOSTnets as gn

In [357]:
sys.path.append(r"/Users/tomgertin/repos/gostrocks/src")
import GOSTRocks.rasterMisc as rMisc
importlib.reload(rMisc)

<module 'GOSTRocks.rasterMisc' from '/Users/tomgertin/repos/gostrocks/src/GOSTRocks/rasterMisc.py'>

In [358]:
sys.path.append(r"/Users/tomgertin/repos/GOSTNets_Raster/src")
import GOSTNetsRaster.market_access as ma
importlib.reload(ma)

<module 'GOSTNetsRaster.market_access' from '/Users/tomgertin/repos/GOSTNets_Raster/src/GOSTNetsRaster/market_access.py'>

In [359]:
global_friction = r"/Users/tomgertin/data/2020_motorized_friction_surface.geotiff"
friction_file = os.path.join(out_folder, global_friction)

In [360]:
#origins_file = r"inputs\bgd_pd_2020_1km.tif"

In [361]:
health_facilities = r"inputs/central_america_health_pts.shp"
#health_facilities = r"inputs/subset_test_health_pts.shp"
health_facilities = gpd.read_file(health_facilities)

In [364]:
out_folder = r"output"
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

In [365]:
#global_friction_clipped = r"/Users/tomgertin/data/2020_motorized_friction_cen_america.geotiff"
global_friction_clipped = r"/Users/tomgertin/data/central_america_min_bounding_box_clipped2.geotiff"

In [367]:
#clip global friction surface to admin area
if not os.path.exists(global_friction_clipped):
    rMisc.clipRaster(rasterio.open(global_friction), gpd.read_file(admin), global_friction_clipped)

## Standardize or Co-register population file to match the friction surface raster

In [368]:
# create friction surface
inR = rasterio.open(global_friction_clipped)

## create friction surface

The actual units within the friction surface raster are minutes required to travel one meter. Therefore multiple by 1000 to get an approximate time in minutes it takes to cross a pixel, because 30-arcsec resolution pixel is approx. 1km by 1km at the equator.

At the equator, an arc-second of longitude approximately equals 30.87 meters. Arc-seconds of longitude decrease in a trigonometric cosine-based fashion as one moves toward the earth's poles (https://www.esri.com/news/arcuser/0400/wdside.html).

In [370]:
# import math

# # COS of 23.5 degrees * length of arc-sec at equator * 30-arcsec
# math.cos(math.radians(23.5)) * 30.87 * 30

#frictionD = inR.read()[0,:,:] * 849
frictionD = inR.read()[0,:,:]
# convert friction surface to traversal time (lazily). Original data are
#    the original data are minutes to travel 1 m, so we will convert to 
#    minutes to cross the cell
frictionD = frictionD * 1000

# Correct no data values
frictionD[frictionD < 0] = 99999999

mcp = graph.MCP_Geometric(frictionD)

# type(frictionD)

# inR.crs

## Calculate Travel Time

In [371]:
import skimage.graph as graph

In [372]:
from skimage.graph import _mcp

In [375]:
#ma.calculate_travel_time?

In [376]:
travel_costs, traceback = ma.calculate_travel_time(inR, mcp, health_facilities, out_raster = os.path.join(out_folder, "cen_america_tt_health.tif"))
                                                   